In [1]:
import torch
from PIL import Image
from pyramid_dit import PyramidDiTForVideoGeneration
from diffusers.utils import load_image, export_to_video
import time

/home/gmacri/miniconda3/envs/pyramid/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
PATH = './local_model'


device = "cuda:1"
torch.cuda.set_device(int(device[-1]))

In [6]:

model_dtype, torch_dtype = 'bf16', torch.bfloat16   # Use bf16 (not support fp16 yet)

model = PyramidDiTForVideoGeneration(
    PATH,                                         # The downloaded checkpoint dir
    model_dtype,
    model_name="pyramid_flux",
    model_variant='diffusion_transformer_384p',     # SD3 supports 'diffusion_transformer_768p'
)

model.vae.enable_tiling()
model.vae.to(device)
model.dit.to(device)
model.text_encoder.to(device)

# if you're not using sequential offloading bellow uncomment the lines above ^
#model.enable_sequential_cpu_offload()

Using temporal causal attention


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.80it/s]
An error occurred while trying to fetch ./local_model/causal_video_vae: Error no file named diffusion_pytorch_model.safetensors found in directory ./local_model/causal_video_vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/gmacri/miniconda3/envs/pyramid/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


The latent dimmension channes is 16
The start sigmas and end sigmas of each stage is Start: {0: 1.0, 1: 0.8002399489209289, 2: 0.5007496155411024}, End: {0: 0.6669999957084656, 1: 0.33399999141693115, 2: 0.0}, Ori_start: {0: 1.0, 1: 0.6669999957084656, 2: 0.33399999141693115}


FluxTextEncoderWithMask(
  (text_encoder): CLIPTextModel(
    (text_model): CLIPTextTransformer(
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 768)
        (position_embedding): Embedding(77, 768)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-11): 12 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=768, out_features=3072, bias=True)
              (fc2): Linear(in_features=3072, out_featu

In [6]:
# first tries with an artistic prompt
art_prompt = "This is a sombre image of violent forces of nature where the expressive feel is intensified by the fierceness of Strindberg’s spatula work. He found the scene depicted at Dalar¨o in the Stockholm skerries in the summer of 1892. Strindberg worked very intuitively, translating his mental state into images with brief, fierce bursts of activity. That is also why he chose small-scale formats; in fact this painting, which Strindberg called The Flying Dutchman as a reference to Richard Wagner’s lonely, ever roaming captain, is one of his largest"

prompt = f"a video derived from the following artwork description '{art_prompt}'"

video_out_base_name = "output_art"

In [4]:
# chpt art

# prompt: i would like to use a text-to-video ai to generate some artistic videos from the descriptions of some paintings. However, using directly the painting description or simply pretending it to be a brief introduction to create a video doesn't seem to be that suitable for this task leading to poor results. So I now give you the original description and I would like you to create one that preserves the meaning but adapts it to the video generation task, creating an artistic video as a result that is rather high pace and dynamic. here is the original description "This is a sombre image of violent forces of nature where the expressive feel is intensified by the fierceness of Strindberg’s spatula work. He found the scene depicted at Dalar¨o in the Stockholm skerries in the summer of 1892. Strindberg worked very intuitively, translating his mental state into images with brief, fierce bursts of activity. That is also why he chose small-scale formats; in fact this painting, which Strindberg called The Flying Dutchman as a reference to Richard Wagner’s lonely, ever roaming captain, is one of his largest"

prompt = "A fierce, storm-lashed seascape unfolds, with towering waves crashing and dark clouds swirling in violent motion. The sky churns in chaotic strokes, capturing the raw power of nature as it collides with an unseen force. Brushstrokes and spatula marks give the scene a visceral intensity, as though painted with a storm's own fury. Quick bursts of flashing light illuminate a lone, spectral ship—a ghostly figure, ever-drifting, embodying the spirit of Wagner’s haunted captain, cursed to roam through darkened waters. In this relentless motion, the scene becomes a tempestuous expression of inner turmoil, echoing the artist’s own struggles. Dark, fleeting glimpses of sky and sea reflect a soul locked in eternal, furious pursuit. A storm that never ends."

video_out_base_name = "output_art_cgpt"

In [7]:
# prompt adjusted with llama3

prompt = "Create a sombre and intense video depicting a turbulent scene of forces of nature, reminiscent of a violent summer storm at Dalar¨o in the Stockholm skerries. Incorporate expressive brushstrokes and vivid colors to convey the emotional turmoil, characteristic of Strindberg's spatula work. Include brief, dynamic bursts of energy, evoking the artist's intuitive and frenzied creative process. Set against a dramatic, isolated backdrop, the scene should evoke the sense of isolation and loneliness, inspired by Richard Wagner's 'Flying Dutchman' legend. Use a mix of fast-paced camera movements and close-ups to capture the raw energy and emotion of the painting."

video_out_base_name = "output_art_llama3"

In [9]:
# prompt adjusted with llama3

prompt = "Create a somber and intense video depicting violent forces of nature, reminiscent of Strindberg's expressive and spontaneous style. Incorporate fierce, bold brushstrokes and vivid colors to convey the energy and turmoil of the scene. Use a mix of fast-paced cuts and slow-motion sequences to capture the dynamic movement of the natural elements. Include a small-scale format, with a focus on bold, expressive lines and vivid textures. Reference the 1892 painting 'The Flying Dutchman' by Strindberg, with a nod to Richard Wagner's iconic character. The tone should evoke a sense of eeriness and foreboding, with a focus on capturing the raw emotion and intensity of the scene."

video_out_base_name =  "output_art_llama3_2"

In [13]:
# prompt adjusted with llama3

prompt = "Create a somber, immersive video showcasing turbulent natural forces, with intense, expressive brushstrokes reminiscent of Strindberg's spatula technique. Depict a dramatic scene set in the Stockholm skerries during summer 1892, inspired by Strindberg's own mental state and translated into dynamic, brief movements. Incorporate a sense of foreboding, using muted colors and bold contrasts to evoke a feeling of unease. Incorporate subtle, suggestive elements to evoke the mythological figure of the Flying Dutchman, as seen in Richard Wagner's work, and explore the contrast between the natural world and the human experience."

video_out_base_name =  "output_art_llama3_3"

In [14]:
with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch_dtype):
    frames = model.generate(
        prompt=prompt,
        num_inference_steps=[20, 20, 20],
        video_num_inference_steps=[10, 10, 10],
        height=384,     
        width=640,
        temp=16,                    # temp=16: 5s, temp=31: 10s
        guidance_scale=7.0,         # The guidance for the first frame, set it to 7 for 384p variant
        video_guidance_scale=5.0,   # The guidance for the other video latent
        output_type="pil",
        save_memory=True,           # If you have enough GPU memory, set it to `False` to improve vae decoding speed
    )

fps = 24

export_to_video(frames, f"{video_out_base_name}_{fps}_{int(time.time())}.mp4", fps=fps)

100%|██████████| 16/16 [01:50<00:00,  6.89s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'output_art_llama3_3_24_1731323270.mp4'

In [7]:
image = Image.open('skerries.jpg').convert("RGB").resize((640, 384))

with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch_dtype):
    frames = model.generate_i2v(
        prompt=prompt,
        input_image=image,
        num_inference_steps=[10, 10, 10],
        temp=16,
        video_guidance_scale=4.0,
        output_type="pil",
        save_memory=True,           # If you have enough GPU memory, set it to `False` to improve vae decoding speed
    )

fps = 24

export_to_video(frames, f"{video_out_base_name}_{fps}_{int(time.time())}_image_conditioned.mp4", fps=fps)

100%|██████████| 15/15 [01:00<00:00,  4.04s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'output_art_24_1730988950_image_conditioned.mp4'

In [11]:
# first tries with an artistic prompt
art_prompt = "Agasse was born in Switzerland, but he lived in London for the last 50 years of his life. His life in London was marked by a great success as an animal painter, and his patrons included many grand landed families. He made something of a speciality in painting exotic animals, and was a frequent visitor to the menageries which were such a feature of London Life at this date"

prompt = f"a video derived from the following artwork description '{art_prompt}'"

video_out_base_name = "output_art_2"

In [14]:

image = Image.open('00136-terriers.jpg').convert("RGB").resize((640, 384))

with torch.no_grad(), torch.cuda.amp.autocast(enabled=True, dtype=torch_dtype):
    frames = model.generate_i2v(
        prompt=prompt,
        input_image=image,
        num_inference_steps=[10, 10, 10],
        temp=16,
        video_guidance_scale=4.0,
        output_type="pil",
        save_memory=True,           # If you have enough GPU memory, set it to `False` to improve vae decoding speed
    )

fps = 24

export_to_video(frames, f"{video_out_base_name}_{fps}_{int(time.time())}_image_conditioned.mp4", fps=fps)

100%|██████████| 15/15 [01:04<00:00,  4.31s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'output_art_2_24_1730991653_image_conditioned.mp4'